In [3]:
from firedrake import *
from gadopt import *
import numpy as np

#set up mesh
rmin, rmax, ncells, nlayers = 1.22, 2.22, 256, 64
bottom_id, top_id = "bottom", "top"

with CheckpointFile("final_state_1e5.h5", 'r') as final_checkpoint:
    mesh = final_checkpoint.load_mesh("firedrake_default_extruded")
    mesh.cartesian = False
    
    T = final_checkpoint.load_function(mesh, "Temperature")
    mu = final_checkpoint.load_function(mesh, "Viscosity")
    # p_load = final_checkpoint.load_function(mesh, "Pressure", idx = 19800)
    # u_load = final_checkpoint.load_function(mesh, "Velocity", idx = 19800)
    # Taverage = final_checkpoint.load_function(mesh, "Average Temperature", idx = 0)

In [15]:
#solving for the Dynamic Topography (DT1)

V = VectorFunctionSpace(mesh, "CG", 2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", 1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", 2)  # Temperature function space (scalar)
Q1 = FunctionSpace(mesh, "CG", 1)  # Average pressure function space (scalar, P1)
Z = MixedFunctionSpace([V, W])  # Mixed function space.

# Paverage = Function(Q1, name="Average Pressure")
# # Calculate the layer average of the initial state
# averager_pressure = LayerAveraging(mesh, np.linspace(rmin, rmax, nlayers * 2), quad_degree=6)
# averager_pressure.extrapolate_layer_average(Paverage, averager_pressure.get_layer_average(p_load))

z = Function(Z)  # A field over the mixed function space Z.
u, p = split(z)  # Returns symbolic UFL expression for u and p

# u_func = u_load 
# p_func = p_load - Paverage

#velocity and pressure functions
u_func, p_func = z.subfunctions
u_func.rename("Velocity")
p_func.rename("Pressure")

Ra = Constant(1e5)  # Rayleigh number
approximation = BoussinesqApproximation(Ra)

Z_nullspace = create_stokes_nullspace(Z, closed=True, rotational=True)

Z_near_nullspace = create_stokes_nullspace(Z, closed=False, rotational=True, translations=[0, 1])

#bcs 
stokes_bcs = {
    bottom_id: {'un': 0},
    top_id: {'un': 0},
}

temp_bcs = {
    bottom_id: {'T': 1.0},
    top_id: {'T': 0.0},
}

stokes_solver = StokesSolver(z, T, approximation, bcs=stokes_bcs, mu=mu,
                            constant_jacobian=True,
                             nullspace=Z_nullspace, transpose_nullspace=Z_nullspace,
                             near_nullspace=Z_near_nullspace)

surface_force_solver = BoundaryNormalStressSolver(stokes_solver, top_id)

# At this point we have all the solver objects we need, we first solve for
# velocity, and then surface force (or surface dynamic topography)

# Solve Stokes sytem:
stokes_solver.solve()
surface_force = surface_force_solver.solve()




# And here we visualise it and write the fields out

VTKFile("DT1.pvd").write(*z.subfunctions, T, surface_force, mu)
with CheckpointFile("dt1.h5", mode="w") as file:
    file.save_mesh(mesh)
    file.save_function(surface_force, name="Surface Force")
    file.save_function(T, name="Temperature")
    file.save_function(mu, name="Viscosity")

  0 SNES Function norm 2.207639994267e+03
      Linear Stokes_fieldsplit_1_ solve converged due to CONVERGED_RTOL iterations 7
  1 SNES Function norm 3.716582048808e-02


In [11]:
#DT2
#solving for the Dynamic Topography (DT2)

from gadopt import *
from gadopt.inverse import *

#load previous stuff - mesh, T, DT1
bottom_id, top_id = "bottom", "top"
with CheckpointFile("dt1.h5", 'r') as final_checkpoint:
    mesh = final_checkpoint.load_mesh("firedrake_default_extruded")
    mesh.cartesian = False
    
    T = final_checkpoint.load_function(mesh, "Temperature")
    rt1 = final_checkpoint.load_function(mesh, "Surface Force")

V = VectorFunctionSpace(mesh, "CG", 2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", 1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", 2)  # Temperature function space (scalar)
Z = MixedFunctionSpace([V, W])  # Mixed function space.

z = Function(Z)  # A field over the mixed function space Z.
u, p = split(z)  # Returns symbolic UFL expression for u and p
z.subfunctions[0].rename("Velocity")
z.subfunctions[1].rename("Pressure")

Ra = Constant(1e5)  # Rayleigh number
approximation = BoussinesqApproximation(Ra)

#viscosity as control d(mu)
mu_control = Function(W, name="control").assign(0.0)
control = Control(mu_control)

#viscosity (isoviscous)
mu_2 = Function(W, name="viscosity")
mu_2.interpolate(10 ** (mu_control))


Z_nullspace = create_stokes_nullspace(Z, closed=True, rotational=True)

Z_near_nullspace = create_stokes_nullspace(Z, closed=False, rotational=True, translations=[0, 1])

#bcs 
stokes_bcs = {
    bottom_id: {'un': 0},
    top_id: {'un': 0},
}

temp_bcs = {
    bottom_id: {'T': 1.0},
    top_id: {'T': 0.0},
}

stokes_solver = StokesSolver(z, T, approximation, bcs=stokes_bcs, mu=mu_2,
                            constant_jacobian=True,
                             nullspace=Z_nullspace, transpose_nullspace=Z_nullspace,
                             near_nullspace=Z_near_nullspace)

surface_force_solver = BoundaryNormalStressSolver(stokes_solver, top_id)

# At this point we have all the solver objects we need, we first solve for
# velocity, and then surface force (or surface dynamic topography)

# Solve Stokes sytem:
stokes_solver.solve()
rt2 = surface_force_solver.solve()  #residual topography for the isoviscous case

# #calculating objective/error function (J)
# objective_func = assemble(0.5 * (rt2 - rt1) ** 2 * ds_t)

VTKFile("DT2.pvd").write(*z.subfunctions, T, rt2, mu_2)
# with CheckpointFile("checkpoint_surface_force_10e5.h5", mode="w") as file:
#     file.save_mesh(mesh)
#     file.save_function(rt1, name="DT_actual")
#     file.save_function(rt2, name="DT_isoviscous")
#     file.save_function(T, name="Temperature")
#     file.save_function(mu, name="Viscosity")
#     file.save_function(mu_2, name="mu_2")

  0 SNES Function norm 2.207639994267e+03
      Linear Stokes_fieldsplit_1_ solve converged due to CONVERGED_RTOL iterations 3
  1 SNES Function norm 3.768585920649e-02


In [12]:
objective_func = assemble(0.5 * (rt1 - rt2) ** 2 * ds_t)

In [13]:
objective_func

9323938.38155149

In [1]:
from firedrake import *
from gadopt import *
import numpy as np

#set up mesh
rmin, rmax, ncells, nlayers = 1.22, 2.22, 256, 64
bottom_id, top_id = "bottom", "top"

with CheckpointFile("Checkpoint_File_1e5_04sept.h5", 'r') as final_checkpoint:
    mesh = final_checkpoint.load_mesh("firedrake_default_extruded")
    mesh.cartesian = False
    
    T = final_checkpoint.load_function(mesh, "Temperature", idx = 19800)
    mu = final_checkpoint.load_function(mesh, "Viscosity", idx = 19800)
    p_load = final_checkpoint.load_function(mesh, "Pressure", idx = 19800)
    u_load = final_checkpoint.load_function(mesh, "Velocity", idx = 19800)
    Taverage = final_checkpoint.load_function(mesh, "Average Temperature", idx = 0)

with CheckpointFile("final_state_1e5.h5", mode="w") as file:
    file.save_mesh(mesh)
    file.save_function(T, name="Temperature")
    file.save_function(mu, name="Viscosity")
    file.save_function(p_load, name="Pressure")
    file.save_function(u_load, name="Velocity")

/home/prahlada/firedrake/lib/python3.10/site-packages/pytools/persistent_dict.py:59: UserWarning: pytools.persistent_dict: unable to import 'siphash24.siphash13', falling back to hashlib.sha256
  warn("pytools.persistent_dict: unable to import 'siphash24.siphash13', "
firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


hello
